In [12]:
# califrais_beam.jl
# STRATÉGIE FINALE (VNS + Merge/Split (corrigé) + "Minimal Trucks")
#
# Dépendances : CSV, DataFrames, Random, Statistics
using CSV
using DataFrames
using Random
using Statistics
import Base.isless, Base.isequal, Base.hash

# -------------------------
# Structures de données
# -------------------------

struct VehicleFamily
    id::Int
    name::String # Ajout du nom pour l'écriture
    capacity::Float64       # kg
    rental_cost::Float64    # €
    fuel_cost::Float64      # € / meter
    radius_cost::Float64    # € / m^2
    speed::Float64          # m/s
    parking_time::Float64   # s
    alpha::Vector{Float64}    # length 4
    beta::Vector{Float64}     # length 4
end

struct Order
    id::Int
    lat::Float64
    lon::Float64
    weight::Float64
    window_start::Union{Nothing, Float64}
    window_end::Union{Nothing, Float64}
    duration::Union{Nothing, Float64}   # delivery duration ℓi
end

struct Route
    family_id::Int
    orders::Vector{Int}   # sequence of order ids (excluding depot 0)
end

mutable struct Solution
    routes::Vector{Route}
    cost::Float64
    Solution() = new(Vector{Route}(), Inf)
end

# For sorting solutions by cost in Beam search
Base.isless(a::Solution, b::Solution) = a.cost < b.cost

# Equality/hash based on route contents (order + family)
Base.isequal(a::Solution, b::Solution) = begin
    if length(a.routes) != length(b.routes) return false end
    # Comparaison simple (peut être lente si les routes sont longues/nombreuses)
    return Set(a.routes) == Set(b.routes)
end

Base.hash(s::Solution, h::UInt = zero(UInt)) = hash(s.routes, h)

# Il faut aussi définir isequal et hash pour Route pour que Set(a.routes) fonctionne
Base.isequal(a::Route, b::Route) = a.family_id == b.family_id && a.orders == b.orders
Base.hash(r::Route, h::UInt = zero(UInt)) = hash(r.family_id, hash(r.orders, h))


struct Instance
    vehicles::Vector{VehicleFamily}
    orders::Dict{Int, Order}   # Map id -> Order (includes depot id=0)
    depot_id::Int
    rho::Float64              # Earth radius ~ 6.371e6 m
    phi0_deg::Float64         # latitude of depot (degrees) for x scaling
    # Pré-calculer les véhicules triés par capacité
    vehicles_sorted_by_capacity::Vector{VehicleFamily}
end

# -------------------------
# Lecture des fichiers
# -------------------------

"""
read_data(vehicles_csv, instance_csv)
Lit vehicles.csv et instance.csv et renvoie Instance.
"""
function read_data(vehicles_csv::String, instance_csv::String)::Instance
    println("Lecture: $vehicles_csv , $instance_csv")
    vdf = CSV.File(vehicles_csv) |> DataFrame
    id = 0
    vehicles = Vector{VehicleFamily}()
    
    for row in eachrow(vdf)
        id += 1
        α = [row[:fourier_cos_0], row[:fourier_cos_1], row[:fourier_cos_2], row[:fourier_cos_3]]
        β = [row[:fourier_sin_0], row[:fourier_sin_1], row[:fourier_sin_2], row[:fourier_sin_3]]
        rental_cost = float(row[:rental_cost])
        
        push!(vehicles, VehicleFamily(id,
                                        string(row[:family]), # Stocker le nom
                                        float(row[:max_capacity]),
                                        rental_cost,
                                        float(row[:fuel_cost]),
                                        float(row[:radius_cost]),
                                        float(row[:speed]),
                                        float(row[:parking_time]),
                                        Float64.(α),
                                        Float64.(β)))
    end

    idmap = Dict{Int, Order}()
    id_rows = CSV.File(instance_csv) |> DataFrame
    depot_lat = nothing
    for row in eachrow(id_rows)
        i = Int(row[:id])
        lat = float(row[:latitude])
        lon = float(row[:longitude])
        if row[:order_weight] === missing
            w = 0.0
            win_s = nothing
            win_e = nothing
            dur = nothing
            depot_lat = lat
        else
            w = float(row[:order_weight])
            win_s = float(row[:window_start])
            win_e = float(row[:window_end])
            dur = float(row[:delivery_duration])
        end
        idmap[i] = Order(i, lat, lon, w, win_s, win_e, dur)
    end

    # Choose phi0 = depot latitude (first depot row). If not found, take mean lat.
    phi0 = isnothing(depot_lat) ? mean([o.lat for o in values(idmap)]) : depot_lat
    # Pré-calculer les véhicules triés
    vehicles_sorted = sort(vehicles, by = vf -> -vf.capacity)
    
    return Instance(vehicles, idmap, 0, 6.371e6, phi0, vehicles_sorted)
end

# -------------------------
# Distances / conversions
# -------------------------

"""
Convertit deux positions géographiques (lat, lon en degrés)
en coordonnées relatives (x, y) en mètres.
"""
function geo_to_xy(inst::Instance, lat1, lon1, lat2, lon2)
    ρ = inst.rho
    # Différences en mètres selon la formule du sujet
    Δy = ρ * (2π / 360) * (lat2 - lat1)
    Δx = ρ * cos((2π / 360) * inst.phi0_deg) * (2π / 360) * (lon2 - lon1)
    return (Δx, Δy)
end

"""
Manhattan δM(i,j) = |xj - xi| + |yj - yi|
"""
function deltaM(inst::Instance, id1::Int, id2::Int)
    o1 = inst.orders[id1]; o2 = inst.orders[id2]
    (dx, dy) = geo_to_xy(inst, o1.lat, o1.lon, o2.lat, o2.lon)
    return abs(dx) + abs(dy)
end

"""
Euclidean δE(i,j) = sqrt((xj - xi)^2 + (yj - yi)^2)
"""
function deltaE(inst::Instance, id1::Int, id2::Int)
    o1 = inst.orders[id1]; o2 = inst.orders[id2]
    (dx, dy) = geo_to_xy(inst, o1.lat, o1.lon, o2.lat, o2.lon)
    return sqrt(dx^2 + dy^2)
end

# -------------------------
# Travel times (τf(i,j|t))
# -------------------------

const T_day = 86400.0
const ω = 2π / T_day

"""
Gamma function γf(t) using Fourier coefficients alpha and beta (n=0..3).
"""
function gamma_f(vf::VehicleFamily, t::Float64)
    s = 0.0
    for n in 0:3
        s += vf.alpha[n+1] * cos(n * ω * t) + vf.beta[n+1] * sin(n * ω * t)
    end
    return s
end

"""
Reference travel time τf(i,j) = δM(i,j)/sf + pf
Travel time at departure t: τf(i,j|t) = τf(i,j) * γf(t)
Assume FIFO (γf non-negative and such).
"""
function travel_time(inst::Instance, vf::VehicleFamily, from::Int, to::Int, depart_time::Float64)
    dM = deltaM(inst, from, to)
    τ_ref = dM / vf.speed + vf.parking_time
    γ = gamma_f(vf, depart_time)
    # Ensure non-negative
    return max(0.0, τ_ref * γ)
end

# -------------------------
# Feasibility and cost
# -------------------------

"""
Simule une route (family, sequence orders) et renvoie (feasible::Bool, route_cost::Float64, end_time::Float64).
Vérifie contraintes fenêtes & calcul partial fuel & radius & rent.
Assume depot id is inst.depot_id (0).
"""
function evaluate_route(inst::Instance, route::Route)
    depot = inst.depot_id
    vf = inst.vehicles[route.family_id]
    seq = route.orders
    
    # Fournir 'init=0.0' au cas où 'seq' est vide
    total_weight = sum((inst.orders[o].weight for o in seq); init=0.0)
    
    if total_weight > vf.capacity + 1e-9
        return (false, Inf, Inf)
    end

    # simulate times
    tdep = 0.0
    current_time = tdep
    prev_node = depot

    # accumulate fuel distance (Manhattan)
    total_manhattan = 0.0

    # For windows, arrival time ak must be within [tmin, tmax]
    for oid in seq
        # travel from prev_node to oid
        # compute travel time using depart time current_time
        τ = travel_time(inst, vf, prev_node, oid, current_time)
        arrival = current_time + τ
        order = inst.orders[oid]
        # If arrival before window_start, wait
        if order.window_start !== nothing && arrival < order.window_start - 1e-9
            arrival = order.window_start
        end
        # Check window upper bound
        if order.window_end !== nothing && arrival > order.window_end + 1e-5
            return (false, Inf, Inf)
        end
        # departure after service
        service_dur = order.duration === nothing ? 0.0 : order.duration
        departure = arrival + service_dur
        # update aggregates
        total_manhattan += deltaM(inst, prev_node, oid)
        current_time = departure
        prev_node = oid
    end

    # return trip to depot (for distances/cost). For times, we don't need to check windows at depot.
    total_manhattan += deltaM(inst, prev_node, depot)

    # cost components
    c_rental = vf.rental_cost
    c_fuel = vf.fuel_cost * total_manhattan

    # radius: max pairwise Euclidean among orders in route (exclude depot)
    radius_val = 0.0
    n = length(seq)
    if n >= 2
        maxd = 0.0
        for i in 1:n-1
            for j in i+1:n
                d = deltaE(inst, seq[i], seq[j])
                if d > maxd
                    maxd = d
                end
            end
        end
        radius_val = (0.5 * maxd)^2
    else
        radius_val = 0.0
    end
    c_radius = vf.radius_cost * radius_val
    total_cost = c_rental + c_fuel + c_radius

    return (true, total_cost, current_time)
end

"""
Evaluate whole solution: verify disjoint coverage (each order once), per-route feasibility,
and compute total cost. If infeasible, return Inf and set solution.cost=Inf.
"""
function evaluate_solution!(sol::Solution, inst::Instance)
    # check that every non-depot order belongs to exactly one route
    all_order_ids = [o.id for o in values(inst.orders) if o.id != inst.depot_id]
    served = Dict{Int,Int}()   # count
    for o in all_order_ids
        served[o] = 0
    end
    
    total_cost = 0.0
    if isempty(sol.routes) && !isempty(all_order_ids)
        sol.cost = Inf # Pas de routes mais des commandes à servir
        return
    end

    for r in sol.routes
        for o in r.orders
            if !haskey(served, o)
                # Commande invalide dans la route (ex: dépôt)
                sol.cost = Inf
                return
            end
            served[o] += 1
        end
    end
    # each served exactly once
    for (o,count) in served
        if count != 1
            sol.cost = Inf
            return
        end
    end

    # evaluate each route
    for r in sol.routes
        feasible, cost_r, _ = evaluate_route(inst, r)
        if !feasible
            sol.cost = Inf
            return
        end
        total_cost += cost_r
    end

    sol.cost = total_cost
end

# ----------------------------------------------------
# Génération solution initiale (MODIFIÉE)
# ----------------------------------------------------

"""
(Stratégie "Minimal Vehicles")
Heuristique "Greedy Insertion" (type "Best Fit")
Tente de minimiser le nombre de routes en remplissant d'abord les routes existantes.
"""
function generate_initial_minimal_trucks(inst::Instance; rng=Random.GLOBAL_RNG)
    sol = Solution()
    
    # 1. Mélanger les commandes pour obtenir des solutions différentes
    orders_to_assign = [o.id for o in values(inst.orders) if o.id != inst.depot_id]
    shuffle!(rng, orders_to_assign)

    for oid in orders_to_assign
        inserted = false
        
        # 2. Essayer d'insérer dans une route existante (meilleure insertion)
        best_route_idx = -1
        best_pos = -1
        best_cost_delta = Inf
            
        for (r_idx, route) in enumerate(sol.routes)
            # Essayer toutes les positions d'insertion
            for pos in 1:(length(route.orders) + 1)
                test_seq = copy(route.orders)
                insert!(test_seq, pos, oid)
                test_route = Route(route.family_id, test_seq)
                (feasible, cost_r, _) = evaluate_route(inst, test_route)
                
                if feasible
                    # Calculer le "delta" (augmentation du coût de la route)
                    (feasible_orig, cost_orig, _) = evaluate_route(inst, route)
                    delta = cost_r - cost_orig
                    if delta < best_cost_delta
                        best_cost_delta = delta
                        best_pos = pos
                        best_route_idx = r_idx
                    end
                end
            end
        end
            
        # Si on a trouvé une position valide, insérer et passer à la commande suivante
        if best_pos != -1
            insert!(sol.routes[best_route_idx].orders, best_pos, oid)
            inserted = true
        end
        
        # 3. Si impossible d'insérer, créer une nouvelle route
        if !inserted
            # Stratégie "Fill Largest" :
            # Essayer de la mettre dans le plus gros camion possible
            new_route_created = false
            for vf in inst.vehicles_sorted_by_capacity # (Pré-trié par capacité)
                test_route = Route(vf.id, [oid])
                (feasible, _, _) = evaluate_route(inst, test_route)
                
                if feasible
                    push!(sol.routes, test_route)
                    new_route_created = true
                    break # On a trouvé un camion
                end
            end

            if !new_route_created
                # Si même une route simple n'est pas faisable (problème de données)
                println("AVERTISSEMENT: Commande $oid infaisable seule. Forçage.")
                push!(sol.routes, Route(inst.vehicles_sorted_by_capacity[1].id, [oid])) # Ajoute la route infaisable
            end
        end
    end
    
    evaluate_solution!(sol, inst)
    return sol
end

"""
Génère k solutions initiales en utilisant la stratégie "Minimal Vehicles".
"""
function generate_k_initial_solutions(k::Int, inst::Instance)
    sols = Set{Solution}() # Utiliser un Set pour garantir k solutions uniques
    attempts = 0
    println("Génération de $k solutions (Minimal Trucks Strategy)...")
    
    while length(sols) < k && attempts < k * 10 # Limiter les tentatives
        # Appelle l'heuristique "Minimal Vehicles"
        s = generate_initial_minimal_trucks(inst; rng=MersenneTwister(rand(UInt)))
        if s.cost != Inf
            push!(sols, s)
        end
        attempts += 1
    end

    if isempty(sols) && attempts > 0
        println("AVERTISSEMENT : Impossible de générer une solution initiale faisable.")
    end
    return collect(sols)
end

"""
Helper : génère m nouvelles solutions aléatoires (stratégie "Minimal Vehicles").
"""
function generate_m_random_solutions(m::Int, inst::Instance)
    sols = Vector{Solution}()
    for _ in 1:m
        # Appelle l'heuristique "Minimal Vehicles"
        s = generate_initial_minimal_trucks(inst; rng=MersenneTwister(rand(UInt)))
        if s.cost != Inf
            push!(sols, s)
        end
    end
    return sols
end

# ----------------------------------------------------
# Voisinages (9 types)
# ----------------------------------------------------

"""
V1: Relocate (Transférer)
Prend une commande au hasard et la déplace vers la *meilleure* autre position.
"""
function move_relocate_best(sol::Solution, inst::Instance)
    if isempty(sol.routes) return nothing end
    src_indices = [i for (i,r) in enumerate(sol.routes) if !isempty(r.orders)]
    if isempty(src_indices) return nothing end
    r_from_idx = rand(src_indices)
    r_from = sol.routes[r_from_idx]
    oid = rand(r_from.orders)

    base_routes = deepcopy(sol.routes)
    # remove oid from source route
    filter!(x -> x != oid, base_routes[r_from_idx].orders)

    candidates = Vector{Solution}()
    # try insert into existing routes
    for (ri, rto) in enumerate(base_routes)
        for pos in 0:length(rto.orders)
            new_routes = deepcopy(base_routes)
            insert!(new_routes[ri].orders, pos+1, oid)
            
            candidate = Solution()
            candidate.routes = new_routes
            
            evaluate_solution!(candidate, inst)
            if candidate.cost != Inf
                push!(candidates, candidate)
            end
        end
    end
    # try new route with any vehicle family
    for vf in inst.vehicles
        new_routes = deepcopy(base_routes)
        push!(new_routes, Route(vf.id, [oid]))
        
        candidate = Solution()
        candidate.routes = new_routes
        
        evaluate_solution!(candidate, inst)
        if candidate.cost != Inf
            push!(candidates, candidate)
        end
    end

    if isempty(candidates) return nothing end
    return minimum(candidates) # best_by_cost
end

"""
V2: Inter-Route Swap (Échange simple)
Échange deux commandes au hasard entre deux routes (meilleur échange).
"""
function move_inter_route_swap(sol::Solution, inst::Instance)
    # pick two distinct routes with at least one order each
    r_indices = [i for (i,r) in enumerate(sol.routes) if !isempty(r.orders)]
    if length(r_indices) < 2 return nothing end # A besoin d'au moins 2 routes non-vides
    
    shuffle!(r_indices)
    i1, i2 = r_indices[1], r_indices[2]
    r1, r2 = sol.routes[i1], sol.routes[i2]
    
    candidates = Vector{Solution}()
    # try all pairs of positions
    for pos1 in 1:length(r1.orders)
        for pos2 in 1:length(r2.orders)
            nroutes = deepcopy(sol.routes)
            o1 = nroutes[i1].orders[pos1]
            o2 = nroutes[i2].orders[pos2]
            nroutes[i1].orders[pos1] = o2
            nroutes[i2].orders[pos2] = o1
            
            candidate = Solution()
            candidate.routes = nroutes
            
            evaluate_solution!(candidate, inst)
            if candidate.cost != Inf
                push!(candidates, candidate)
            end
        end
    end
    if isempty(candidates) return nothing end
    return minimum(candidates) # best_by_cost
end

"""
V3: Intra-Route Swap (Échange interne)
Échange deux commandes au hasard *à l'intérieur* de la même route (meilleur échange).
"""
function move_intra_route_swap(sol::Solution, inst::Instance)
    r_indices = [i for (i,r) in enumerate(sol.routes) if length(r.orders) >= 2]
    if isempty(r_indices) return nothing end
    
    r_idx = rand(r_indices)
    route = sol.routes[r_idx]
    n = length(route.orders)
    
    candidates = Vector{Solution}()
    for i in 1:n-1
        for j in i+1:n
            nroutes = deepcopy(sol.routes)
            o_i = nroutes[r_idx].orders[i]
            o_j = nroutes[r_idx].orders[j]
            nroutes[r_idx].orders[i] = o_j
            nroutes[r_idx].orders[j] = o_i
            
            candidate = Solution()
            candidate.routes = nroutes
            evaluate_solution!(candidate, inst)
            if candidate.cost != Inf
                push!(candidates, candidate)
            end
        end
    end
    if isempty(candidates) return nothing end
    return minimum(candidates) # best_by_cost
end


"""
V4: Change Vehicle (Simple)
Prend une route au hasard et change sa famille de véhicule pour une autre.
"""
function move_change_vehicle(sol::Solution, inst::Instance)
    if isempty(sol.routes) return nothing end
    
    # 1. Pick a random route
    r_idx = rand(1:length(sol.routes))
    route_to_change = sol.routes[r_idx]
    
    # 2. Pick a *different* vehicle family
    current_family_id = route_to_change.family_id
    other_family_ids = [vf.id for vf in inst.vehicles if vf.id != current_family_id]
    if isempty(other_family_ids) return nothing end # Only one vehicle type
    
    new_family_id = rand(other_family_ids)
    
    # 3. Create the new route
    new_route = Route(new_family_id, route_to_change.orders)
    
    # 4. Check feasibility of *this route*
    (feasible, cost_r, _) = evaluate_route(inst, new_route)
    
    if !feasible
        return nothing
    end
    
    # 5. Create new solution and evaluate
    new_sol = Solution()
    new_sol.routes = deepcopy(sol.routes)
    new_sol.routes[r_idx] = new_route # Replace the old route
    
    evaluate_solution!(new_sol, inst) # Recalculate total cost
    
    if new_sol.cost == Inf
        return nothing # Should be caught by evaluate_route, but as a safeguard
    end
    
    return new_sol
end

"""
V5: Multi-Inter-Route Swap (Variable)
Effectue `n_swaps = rand(1:n_max_swaps)` échanges aléatoires entre les routes.
"""
function move_multi_inter_route_swap(sol::Solution, inst::Instance; n_max_swaps::Int=3)
    new_sol = deepcopy(sol)
    
    # Trouver les routes qui ont au moins une commande
    valid_route_indices = [i for (i,r) in enumerate(new_sol.routes) if !isempty(r.orders)]
    if length(valid_route_indices) < 2
        return nothing # Pas assez de routes pour échanger
    end

    n_swaps_to_do = rand(1:n_max_swaps) # Choisir n au hasard

    for _ in 1:n_swaps_to_do
        # Choisir deux routes non-vides différentes
        i1, i2 = rand(valid_route_indices, 2)
        if i1 == i2 continue end 
        
        r1, r2 = new_sol.routes[i1], new_sol.routes[i2]
        
        # S'assurer qu'elles ne sont pas vides (elles pourraient l'être devenues)
        if isempty(r1.orders) || isempty(r2.orders) continue end
        
        # Choisir les positions à échanger
        pos1 = rand(1:length(r1.orders))
        pos2 = rand(1:length(r2.orders))
        
        # Echanger
        o1 = r1.orders[pos1]
        o2 = r2.orders[pos2]
        r1.orders[pos1] = o2
        r2.orders[pos2] = o1
    end
    
    evaluate_solution!(new_sol, inst)
    return (new_sol.cost != Inf) ? new_sol : nothing
end

"""
V6: Multi-Intra-Route Swap (Mélange)
Effectue `n_swaps = rand(1:n_max_swaps)` échanges aléatoires *à l'intérieur* d'une route.
"""
function move_multi_intra_route_swap(sol::Solution, inst::Instance; n_max_swaps::Int=3)
    new_sol = deepcopy(sol)
    r_indices = [i for (i,r) in enumerate(new_sol.routes) if length(r.orders) >= 2]
    if isempty(r_indices) return nothing end
    
    r_idx = rand(r_indices)
    route_orders = new_sol.routes[r_idx].orders
    n = length(route_orders)
    
    n_swaps_to_do = rand(1:n_max_swaps)
    
    for _ in 1:n_swaps_to_do
        pos1 = rand(1:n)
        pos2 = rand(1:n)
        if pos1 == pos2 continue end
        
        o1 = route_orders[pos1]
        o2 = route_orders[pos2]
        route_orders[pos1] = o2
        route_orders[pos2] = o1
    end
    
    evaluate_solution!(new_sol, inst)
    return (new_sol.cost != Inf) ? new_sol : nothing
end

"""
V7: Multi-Change Vehicle (Variable)
Effectue `n_changes = rand(1:n_max_changes)` changements de véhicule aléatoires.
"""
function move_multi_change_vehicle(sol::Solution, inst::Instance; n_max_changes::Int=3)
    if isempty(sol.routes) || length(inst.vehicles) < 2 return nothing end
    new_sol = deepcopy(sol)
    
    route_indices = 1:length(new_sol.routes)
    vehicle_ids = [v.id for v in inst.vehicles]

    n_changes_to_do = rand(1:n_max_changes) # Choisir n au hasard

    for _ in 1:n_changes_to_do
        r_idx = rand(route_indices)
        current_family_id = new_sol.routes[r_idx].family_id
        
        # Choisir une famille différente
        new_family_id = rand(vehicle_ids)
        while new_family_id == current_family_id
             # Gérer le cas où il n'y a qu'un seul type de véhicule
             if length(vehicle_ids) == 1 break end
             new_family_id = rand(vehicle_ids)
        end
        
        new_sol.routes[r_idx] = Route(new_family_id, new_sol.routes[r_idx].orders)
    end
    
    evaluate_solution!(new_sol, inst)
    return (new_sol.cost != Inf) ? new_sol : nothing
end

"""
V8: Split Route (Séparer)
Prend une route avec au moins 4 commandes et la sépare en deux.
"""
function move_split_route(sol::Solution, inst::Instance)
    # Choisir une route candidate (au moins 4 commandes pour un split intéressant)
    r_indices = [i for (i,r) in enumerate(sol.routes) if length(r.orders) >= 4]
    if isempty(r_indices) return nothing end
    
    r_idx = rand(r_indices)
    route_to_split = sol.routes[r_idx]
    n = length(route_to_split.orders)
    
    # Choisir un point de split (au moins 2 commandes par nouvelle route)
    split_point = rand(2:(n-2))
    
    # Créer les deux nouvelles routes
    orders1 = route_to_split.orders[1:split_point]
    orders2 = route_to_split.orders[split_point+1:end]
    
    # Utiliser le même type de véhicule (on pourrait optimiser ce choix)
    fam_id = route_to_split.family_id
    r1_new = Route(fam_id, orders1)
    r2_new = Route(fam_id, orders2)

    # Vérifier la faisabilité des *deux* nouvelles routes
    (feasible1, cost1, _) = evaluate_route(inst, r1_new)
    if !feasible1 return nothing end
    
    (feasible2, cost2, _) = evaluate_route(inst, r2_new)
    if !feasible2 return nothing end
    
    # Créer la nouvelle solution
    new_sol = Solution()
    new_sol.routes = deepcopy(sol.routes)
    deleteat!(new_sol.routes, r_idx) # Supprimer l'ancienne route
    push!(new_sol.routes, r1_new)    # Ajouter la première partie
    push!(new_sol.routes, r2_new)    # Ajouter la seconde partie
    
    evaluate_solution!(new_sol, inst) # Recalculer le coût total
    return (new_sol.cost != Inf) ? new_sol : nothing
end

"""
V9: Merge Routes (Fusionner) - CORRIGÉ (Randomisé)
Tente de fusionner une route aléatoire (source) dans une autre (destination).
"""
function move_merge_routes(sol::Solution, inst::Instance)
    if length(sol.routes) < 2 return nothing end
    
    # 1. Choisir deux routes *aléatoires* distinctes
    idx_pool = 1:length(sol.routes)
    if length(idx_pool) < 2 return nothing end
    
    r_src_idx, r_dest_idx = rand(idx_pool, 2)
    while r_src_idx == r_dest_idx
        r_dest_idx = rand(idx_pool)
    end
    
    r_src = sol.routes[r_src_idx]
    r_dest = sol.routes[r_dest_idx]
    
    if isempty(r_src.orders) return nothing end
    
    temp_route_orders = deepcopy(r_dest.orders)
    
    # 2. Essayer d'insérer chaque commande de r_src dans r_dest (best-fit)
    all_inserted = true
    for oid in r_src.orders
        best_pos = -1
        best_cost_delta = Inf
        
        # Sauvegarder la séquence actuelle pour les tests
        current_orders_for_test = copy(temp_route_orders)
        
        for pos in 1:(length(current_orders_for_test) + 1)
            test_seq = copy(current_orders_for_test)
            insert!(test_seq, pos, oid)
            test_route = Route(r_dest.family_id, test_seq)
            
            (feasible, cost_r, _) = evaluate_route(inst, test_route)
            if feasible
                # Calculer le "delta" (augmentation du coût de la route)
                (feasible_orig, cost_orig, _) = evaluate_route(inst, Route(r_dest.family_id, current_orders_for_test))
                delta = cost_r - cost_orig
                if delta < best_cost_delta
                    best_cost_delta = delta
                    best_pos = pos
                end
            end
        end
        
        if best_pos != -1
            # Insérer pour de bon dans la route temporaire
            insert!(temp_route_orders, best_pos, oid)
        else
            # Echec de l'insertion, la fusion est impossible
            all_inserted = false
            break
        end
    end
    
    if !all_inserted
        return nothing
    end
    
    # 3. Si on arrive ici, la fusion a réussi !
    new_sol = Solution()
    new_sol.routes = deepcopy(sol.routes)
    
    # Remplacer la route destination
    new_sol.routes[r_dest_idx] = Route(r_dest.family_id, temp_route_orders)
    # Supprimer la route source (attention aux indices !)
    # Il faut supprimer le plus grand indice en premier
    if r_src_idx > r_dest_idx
        deleteat!(new_sol.routes, r_src_idx)
    else
        deleteat!(new_sol.routes, r_src_idx)
    end
    
    evaluate_solution!(new_sol, inst)
    return (new_sol.cost != Inf) ? new_sol : nothing
end

# -------------------------
# Beam Search Optimizer (Modifié)
# -------------------------

"""
beam_search_optimizer(...)
Applique la nouvelle stratégie de Beam Search avec 9 voisinages + aléatoire.
"""
function beam_search_optimizer(inst::Instance; 
                               k::Int=10, 
                               max_iterations::Int=200, 
                               m_neighbors::Int=2, 
                               m_random::Int=2,
                               n_max_multi_swaps::Int=3,
                               n_max_multi_intra_swaps::Int=3,
                               n_max_multi_changes::Int=3)
                               
    println("Démarrage Beam Search: k=$k, max_iter=$max_iterations, m_neighbors=$m_neighbors, m_random=$m_random")
    println("Voisinages Multi (max n): InterSwap=$n_max_multi_swaps, IntraSwap=$n_max_multi_intra_swaps, ChangeVeh=$n_max_multi_changes")

    # 1. Générer k solutions initiales faisables (stratégie diverse)
    init_sols = generate_k_initial_solutions(k, inst)
    if isempty(init_sols)
        println("Aucune solution initiale faisable trouvée.")
        return Solution()
    end

    current_solutions = Set{Solution}(init_sols)
    best_solution = minimum(collect(current_solutions))
    println("Coût initial (meilleur): $(best_solution.cost)")

    for iter in 1:max_iterations
        println("--- Itération $iter / $max_iterations ---")
        candidate_pool = Set{Solution}(current_solutions)   # Elitisme

        # 2. Générer m voisins pour chaque type de voisinage
        println("  Génération de voisins...")
        for s in current_solutions
            
            # --- Voisinages SIMPLES (Intensification) ---
            for _ in 1:m_neighbors
                nb = move_relocate_best(s, inst) # 1. Relocate
                if nb !== nothing && nb.cost != Inf; push!(candidate_pool, nb); end
                
                nb = move_inter_route_swap(s, inst) # 2. Inter-Swap
                if nb !== nothing && nb.cost != Inf; push!(candidate_pool, nb); end

                nb = move_intra_route_swap(s, inst) # 3. Intra-Swap
                if nb !== nothing && nb.cost != Inf; push!(candidate_pool, nb); end

                nb = move_change_vehicle(s, inst) # 4. Change Vehicle
                if nb !== nothing && nb.cost != Inf; push!(candidate_pool, nb); end
            end
            
            # --- Voisinages LARGES (Diversification) ---
            for _ in 1:m_neighbors
                nb = move_multi_inter_route_swap(s, inst, n_max_swaps=n_max_multi_swaps) # 5. Multi-Inter-Swap
                if nb !== nothing && nb.cost != Inf; push!(candidate_pool, nb); end
                
                nb = move_multi_intra_route_swap(s, inst, n_max_swaps=n_max_multi_intra_swaps) # 6. Multi-Intra-Swap
                if nb !== nothing && nb.cost != Inf; push!(candidate_pool, nb); end

                nb = move_multi_change_vehicle(s, inst, n_max_changes=n_max_multi_changes) # 7. Multi-Change Vehicle
                if nb !== nothing && nb.cost != Inf; push!(candidate_pool, nb); end
            end
            
            # --- Voisinages STRUCTURELS (Merge/Split) ---
            for _ in 1:m_neighbors
                nb = move_split_route(s, inst) # 8. Split Route
                if nb !== nothing && nb.cost != Inf; push!(candidate_pool, nb); end
                
                nb = move_merge_routes(s, inst) # 9. Merge Routes (Randomisé)
                if nb !== nothing && nb.cost != Inf; push!(candidate_pool, nb); end
            end
        end
        
        # 3. Ajouter m solutions aléatoires (Diversification)
        println("  Ajout de $m_random solutions aléatoires...")
        random_sols = generate_m_random_solutions(m_random, inst)
        union!(candidate_pool, random_sols)


        # 4. Sélectionner les k meilleures
        cand_list = collect(candidate_pool)
        sort!(cand_list)
        new_set = Set{Solution}()
        for sol in cand_list
            if sol.cost != Inf
                push!(new_set, sol)
            end
            if length(new_set) >= k
                break
            end
        end

        if isempty(new_set)
            println("Aucun candidat faisable trouvé cette itération.")
            break
        end

        current_solutions = new_set
        current_best = minimum(collect(current_solutions))
        if current_best.cost < best_solution.cost
            best_solution = current_best
            println("Nouveau meilleur coût: $(best_solution.cost) à l'itération $iter")
        else
            println("Meilleur courant: $(current_best.cost) (pas d'amélioration)")
        end
    end

    println("Beam Search terminé. Meilleur coût final: $(best_solution.cost)")
    return best_solution
end

# -------------------------
# Écriture solution routes.csv
# -------------------------

"""
Write routes.csv as requested:
columns: family, order_1, order_2, ..., order_N
Routes shorter than N padded with empty cells.
"""
function write_routes_csv(sol::Solution, inst::Instance, outpath::String)
    println("Écriture routes dans $outpath")
    # find max route length
    N = maximum([length(r.orders) for r in sol.routes]; init=0)
    
    # Préparer les données pour le DataFrame
    data_dict = Dict{Symbol, Vector{Any}}()
    
    # Obtenir les noms de famille
    family_names = [inst.vehicles[r.family_id].name for r in sol.routes]
    data_dict[:family] = family_names
    
    for k in 1:N
        colname = Symbol("order_$k")
        col_data = Vector{Union{Missing, Int}}()
        for r in sol.routes
            if k <= length(r.orders)
                push!(col_data, r.orders[k])
            else
                push!(col_data, missing)
            end
        end
        data_dict[colname] = col_data
    end
    
    df = DataFrame(data_dict)
    CSV.write(outpath, df)
end


function main()
    # Params (modifier si besoin)
    VEHICLES_FILE = "instances/vehicles.csv"
    INSTANCE_FILE = "instances/instance_10.csv"
    OUTPUT_FILE = "output/routes_10.csv"

    # --- NOUVEAUX PARAMÈTRES DE BEAM ---
    K_BEAMS = 5      # k: Garder les k meilleures
    MAX_ITER = 20    # Nb itérations
    M_NEIGHBORS = 2   # m: Nb. voisins par *chaque* type (Total: 9*2=18 par solution)
    M_RANDOM = 2      # m_random: Nb. solutions aléatoires par itération
    
    # Paramètres 'n' pour les nouveaux mouvements "multi"
    N_MAX_MULTI_SWAPS = 3   # n_max pour les multi-échanges (rand(1:3) swaps)
    N_MAX_MULTI_INTRA_SWAPS = 3 # n_max pour les mélanges de route (rand(1:3) swaps)
    N_MAX_MULTI_CHANGES = 3 # n_max pour les multi-changements (rand(1:2) changes)
    # -----------------------------------

    if !isfile(VEHICLES_FILE) || !isfile(INSTANCE_FILE)
        println("Fichiers d'entrée introuvables. Modifie VEHICLES_FILE / INSTANCE_FILE dans main().")
        return
    end
    
    # Créer le dossier output s'il n'existe pas
    mkpath(dirname(OUTPUT_FILE))

    inst = read_data(VEHICLES_FILE, INSTANCE_FILE)
    println("Instance lue: $(length(inst.vehicles)) familles, $(length(inst.orders)-1) commandes (excl. dépôt).")

    best = beam_search_optimizer(inst, 
                                k=K_BEAMS, 
                                max_iterations=MAX_ITER, 
                                m_neighbors=M_NEIGHBORS, 
                                m_random=M_RANDOM,
                                n_max_multi_swaps=N_MAX_MULTI_SWAPS,
                                n_max_multi_intra_swaps=N_MAX_MULTI_INTRA_SWAPS,
                                n_max_multi_changes=N_MAX_MULTI_CHANGES)

    if best.cost == Inf
        println("Aucune solution faisable trouvée. Aucun fichier écrit.")
    else
        write_routes_csv(best, inst, OUTPUT_FILE)
        println("Solution écrite dans $OUTPUT_FILE — coût: $(best.cost)")
    end
end

main()

Lecture: instances/vehicles.csv , instances/instance_10.csv
Instance lue: 3 familles, 496 commandes (excl. dépôt).
Démarrage Beam Search: k=5, max_iter=20, m_neighbors=2, m_random=2
Voisinages Multi (max n): InterSwap=3, IntraSwap=3, ChangeVeh=3
Génération de 5 solutions (Minimal Trucks Strategy)...
Coût initial (meilleur): 1.9663275046897586e6
--- Itération 1 / 20 ---
  Génération de voisins...
  Ajout de 2 solutions aléatoires...
Nouveau meilleur coût: 1.9575665526415023e6 à l'itération 1
--- Itération 2 / 20 ---
  Génération de voisins...
  Ajout de 2 solutions aléatoires...
Nouveau meilleur coût: 1.921791031021477e6 à l'itération 2
--- Itération 3 / 20 ---
  Génération de voisins...
  Ajout de 2 solutions aléatoires...
Nouveau meilleur coût: 1.8999697075630794e6 à l'itération 3
--- Itération 4 / 20 ---
  Génération de voisins...
  Ajout de 2 solutions aléatoires...
Nouveau meilleur coût: 1.846965640137064e6 à l'itération 4
--- Itération 5 / 20 ---
  Génération de voisins...
  Ajout

In [10]:
function generate_all_neighbors(s::Solution, inst::Instance;
                                n_max_multi_swaps::Int=3,
                                n_max_multi_intra_swaps::Int=3,
                                n_max_multi_changes::Int=3)
    
    neighbors = Set{Solution}()
    
    # --- Voisinages SIMPLES ---
    nb = move_relocate_best(s, inst)
    if nb !== nothing && nb.cost != Inf; push!(neighbors, nb); end
    
    nb = move_inter_route_swap(s, inst)
    if nb !== nothing && nb.cost != Inf; push!(neighbors, nb); end
    
    nb = move_intra_route_swap(s, inst)
    if nb !== nothing && nb.cost != Inf; push!(neighbors, nb); end
    
    nb = move_change_vehicle(s, inst)
    if nb !== nothing && nb.cost != Inf; push!(neighbors, nb); end
    
    # --- Voisinages LARGES ---
    nb = move_multi_inter_route_swap(s, inst, n_max_swaps=n_max_multi_swaps)
    if nb !== nothing && nb.cost != Inf; push!(neighbors, nb); end
    
    nb = move_multi_intra_route_swap(s, inst, n_max_swaps=n_max_multi_intra_swaps)
    if nb !== nothing && nb.cost != Inf; push!(neighbors, nb); end
    
    nb = move_multi_change_vehicle(s, inst, n_max_changes=n_max_multi_changes)
    if nb !== nothing && nb.cost != Inf; push!(neighbors, nb); end
    
    # --- Voisinages STRUCTURELS ---
    nb = move_split_route(s, inst)
    if nb !== nothing && nb.cost != Inf; push!(neighbors, nb); end
    
    nb = move_merge_routes(s, inst)
    if nb !== nothing && nb.cost != Inf; push!(neighbors, nb); end
    
    return neighbors
end

function refine_solution_with_beam(
    starting_solution::Solution,
    inst::Instance; 
    k::Int=10, 
    max_iterations::Int=200, 
    m_neighbors::Int=2, 
    m_random::Int=2,
    n_max_multi_swaps::Int=3,
    n_max_multi_intra_swaps::Int=3,
    n_max_multi_changes::Int=3
)
    
    println("Démarrage du raffinage Beam Search à partir d'une solution existante...")
    println("Coût de la solution de départ: $(starting_solution.cost)")

    # 1. Générer le "beam" initial à partir de la solution et ses voisins
    if starting_solution.cost == Inf
         println("ERREUR: La solution de départ est infaisable.")
         return starting_solution
    end

    init_sols_set = Set{Solution}()
    push!(init_sols_set, starting_solution) # Ajouter la solution elle-même
    
    # Générer les voisins
    println("   Génération des voisins de la solution de départ...")
    initial_neighbors = generate_all_neighbors(
        starting_solution, inst,
        n_max_multi_swaps=n_max_multi_swaps,
        n_max_multi_intra_swaps=n_max_multi_intra_swaps,
        n_max_multi_changes=n_max_multi_changes
    )
    union!(init_sols_set, initial_neighbors)
    println("   $(length(init_sols_set)) solutions initiales (départ + voisins) générées.")

    # Trier et garder les k meilleures pour le "beam" initial
    init_sols_list = sort(collect(init_sols_set))
    
    current_solutions = Set{Solution}()
    for sol in init_sols_list
         if length(current_solutions) < k
             push!(current_solutions, sol)
         else
             break
         end
    end
    
    if isempty(current_solutions)
         println("Aucune solution initiale faisable (même la solution de départ?).")
         return Solution()
    end

    best_solution = minimum(collect(current_solutions))
    println("Meilleur coût après voisinage initial: $(best_solution.cost)")

    # 2. Boucle principale (identique à beam_search_optimizer)
    # ---------------------------------------------------------
    for iter in 1:max_iterations
        println("--- Itération de raffinage $iter / $max_iterations ---")
        candidate_pool = Set{Solution}(current_solutions) # Elitisme

        # 2a. Générer m voisins pour chaque type de voisinage
        println("  Génération de voisins...")
        for s in current_solutions
            # --- Voisinages SIMPLES (Intensification) ---
            for _ in 1:m_neighbors
                nb = move_relocate_best(s, inst)
                if nb !== nothing && nb.cost != Inf; push!(candidate_pool, nb); end
                
                nb = move_inter_route_swap(s, inst)
                if nb !== nothing && nb.cost != Inf; push!(candidate_pool, nb); end

                nb = move_intra_route_swap(s, inst)
                if nb !== nothing && nb.cost != Inf; push!(candidate_pool, nb); end

                nb = move_change_vehicle(s, inst)
                if nb !== nothing && nb.cost != Inf; push!(candidate_pool, nb); end
            end
            
            # --- Voisinages LARGES (Diversification) ---
            for _ in 1:m_neighbors
                nb = move_multi_inter_route_swap(s, inst, n_max_swaps=n_max_multi_swaps)
                if nb !== nothing && nb.cost != Inf; push!(candidate_pool, nb); end
                
                nb = move_multi_intra_route_swap(s, inst, n_max_swaps=n_max_multi_intra_swaps)
                if nb !== nothing && nb.cost != Inf; push!(candidate_pool, nb); end

                nb = move_multi_change_vehicle(s, inst, n_max_changes=n_max_multi_changes)
                if nb !== nothing && nb.cost != Inf; push!(candidate_pool, nb); end
            end
            
            # --- Voisinages STRUCTURELS (Merge/Split) ---
            for _ in 1:m_neighbors
                nb = move_split_route(s, inst)
                if nb !== nothing && nb.cost != Inf; push!(candidate_pool, nb); end
                
                nb = move_merge_routes(s, inst)
                if nb !== nothing && nb.cost != Inf; push!(candidate_pool, nb); end
            end
        end
        
        # 2b. Ajouter m solutions aléatoires (Diversification)
        println("  Ajout de $m_random solutions aléatoires...")
        random_sols = generate_m_random_solutions(m_random, inst)
        union!(candidate_pool, random_sols)

        # 2c. Sélectionner les k meilleures
        cand_list = collect(candidate_pool)
        sort!(cand_list)
        new_set = Set{Solution}()
        for sol in cand_list
            if sol.cost != Inf
                push!(new_set, sol)
            end
            if length(new_set) >= k
                break
            end
        end

        if isempty(new_set)
            println("Aucun candidat faisable trouvé cette itération.")
            break
        end

        current_solutions = new_set
        current_best = minimum(collect(current_solutions))
        if current_best.cost < best_solution.cost
            best_solution = current_best
            println("Nouveau meilleur coût: $(best_solution.cost) à l'itération $iter")
        else
            println("Meilleur courant: $(current_best.cost) (pas d'amélioration)")
        end
    end

    println("Raffinage Beam Search terminé. Meilleur coût final: $(best_solution.cost)")
    return best_solution
end

"""
[NOUVELLE FONCTION DE LECTURE]
Lit un fichier 'routes.csv' (précédemment généré) et le convertit
en un objet Solution.

Requiert l'instance pour mapper les noms de famille (String)
aux ID de famille (Int).
"""
function read_solution_csv(solution_csv::String, inst::Instance)::Solution
    println("Lecture de la solution de départ depuis: $solution_csv")
    sol = Solution()
    
    if !isfile(solution_csv)
        println("ERREUR: Fichier solution '$solution_csv' introuvable.")
        sol.cost = Inf # Marquer comme infaisable
        return sol
    end

    # 1. Créer un mapping "Nom de famille" -> "ID"
    family_name_to_id = Dict{String, Int}()
    for v in inst.vehicles
        family_name_to_id[v.name] = v.id
    end

    # 2. Lire le CSV
    try
        df = CSV.File(solution_csv) |> DataFrame
        
        # 3. Trouver les colonnes 'order_k' et les trier numériquement
        col_names_str = names(df)
        order_cols_str = filter(n -> startswith(n, "order_"), col_names_str)
        
        # Tri numérique (pour que order_10 vienne après order_9)
        order_cols_sym = [Symbol(s) for s in order_cols_str]
        sort!(order_cols_sym, by = c -> parse(Int, split(string(c), '_')[2]))

        # 4. Convertir chaque ligne du DataFrame en Route
        for row in eachrow(df)
            family_name = string(row[:family])
            
            if !haskey(family_name_to_id, family_name)
                println("ERREUR: Famille '$family_name' du CSV non trouvée dans l'instance 'vehicles.csv'.")
                sol.cost = Inf
                return sol # Impossible de continuer
            end
            
            family_id = family_name_to_id[family_name]
            
            orders = Vector{Int}()
            for col in order_cols_sym
                order_id = row[col]
                if !ismissing(order_id)
                    push!(orders, Int(order_id))
                end
            end
            
            # Créer et ajouter la route
            push!(sol.routes, Route(family_id, orders))
        end
        
    catch e
        println("ERREUR lors de la lecture du fichier CSV: $e")
        sol.cost = Inf
        return sol
    end

    # 5. Évaluer la solution chargée pour obtenir son coût
    evaluate_solution!(sol, inst)
    
    if sol.cost == Inf
        println("AVERTISSEMENT: La solution chargée depuis '$solution_csv' est infaisable (coût Inf).")
    else
        println("Solution chargée avec succès. Coût initial: $(sol.cost)")
    end
    
    return sol
end


"""
Exemple d'utilisation : Charger une solution depuis un fichier
et la raffiner.
"""
function main_refinement_from_file_example()
    # --- Fichiers (MODIFIEZ CECI) ---
    VEHICLES_FILE = "instances/vehicles.csv"
    INSTANCE_FILE = "instances/instance_08.csv"
    
    # Le fichier de solution dont vous voulez repartir
    STARTING_SOLUTION_FILE = "output/routes_08.csv" 
    
    # Le fichier où sera sauvegardé le résultat raffiné
    OUTPUT_FILE = "output/routes_08_refined_from_file.csv" 
    
    # --- Paramètres de BEAM (identiques) ---
    K_BEAMS = 10
    MAX_ITER = 20 
    M_NEIGHBORS = 2 
    M_RANDOM = 2 
    N_MAX_MULTI_SWAPS = 5
    N_MAX_MULTI_INTRA_SWAPS = 5
    N_MAX_MULTI_CHANGES = 5
    # --------------------------

    # 1. Vérifier les fichiers d'entrée
    if !isfile(VEHICLES_FILE) || !isfile(INSTANCE_FILE) || !isfile(STARTING_SOLUTION_FILE)
        println("Fichiers d'entrée introuvables. Vérifiez les chemins.")
        println("  VEHICLES_FILE: $VEHICLES_FILE")
        println("  INSTANCE_FILE: $INSTANCE_FILE")
        println("  STARTING_SOLUTION_FILE: $STARTING_SOLUTION_FILE")
        return
    end
    
    mkpath(dirname(OUTPUT_FILE))
    
    # 2. Lire l'instance
    inst = read_data(VEHICLES_FILE, INSTANCE_FILE)
    println("Instance lue: $(length(inst.vehicles)) familles, $(length(inst.orders)-1) commandes.")

    # 3. Lire la solution de DÉPART depuis le fichier CSV
    starting_sol = read_solution_csv(STARTING_SOLUTION_FILE, inst)
    
    if starting_sol.cost == Inf
        println("Impossible de continuer car la solution de départ est infaisable ou n'a pas pu être lue.")
        return
    end
    
    # 4. Lancer le raffinage (en utilisant la fonction de la cellule précédente)
    println("\n--- Lancement du raffinage de la solution chargée ---")
    refined_best = refine_solution_with_beam(
        starting_sol, inst, 
        k=K_BEAMS, 
        max_iterations=MAX_ITER, 
        m_neighbors=M_NEIGHBORS, 
        m_random=M_RANDOM,
        n_max_multi_swaps=N_MAX_MULTI_SWAPS,
        n_max_multi_intra_swaps=N_MAX_MULTI_INTRA_SWAPS,
        n_max_multi_changes=N_MAX_MULTI_CHANGES
    )

    # 5. Écrire le résultat
    if refined_best.cost == Inf
        println("Aucune solution faisable trouvée après raffinage.")
    else
        write_routes_csv(refined_best, inst, OUTPUT_FILE)
        println("Solution raffinée écrite dans $OUTPUT_FILE — coût: $(refined_best.cost)")
    end
end

# Décommentez la ligne ci-dessous pour exécuter l'exemple
main_refinement_from_file_example()

Lecture: instances/vehicles.csv , instances/instance_08.csv
Instance lue: 3 familles, 323 commandes.
Lecture de la solution de départ depuis: output/routes_08.csv
Solution chargée avec succès. Coût initial: 296400.25816320325

--- Lancement du raffinage de la solution chargée ---
Démarrage du raffinage Beam Search à partir d'une solution existante...
Coût de la solution de départ: 296400.25816320325
   Génération des voisins de la solution de départ...
   8 solutions initiales (départ + voisins) générées.
Meilleur coût après voisinage initial: 296400.25816320325
--- Itération de raffinage 1 / 20 ---
  Génération de voisins...
  Ajout de 2 solutions aléatoires...
Nouveau meilleur coût: 295252.55173629214 à l'itération 1
--- Itération de raffinage 2 / 20 ---
  Génération de voisins...
  Ajout de 2 solutions aléatoires...
Nouveau meilleur coût: 294320.4120424598 à l'itération 2
--- Itération de raffinage 3 / 20 ---
  Génération de voisins...
  Ajout de 2 solutions aléatoires...
Nouveau me